## This notebook was only created to test the ReadAndRunAllWorkFlows.py


<br>
<br>

In [1]:
#pull the ara results and look through KF+G edge graphs and pull the povenance attribute and make a table that will look like
#rows will be kp
#col be workflows
#entry would be the ara's that had edges in kp in the workflow. 

In [347]:
import requests
import json
from collections import defaultdict
import os
import pandas as pd


#https://ars.ci.transltr.io/ars/api --- 
#https://arax.ci.transltr.io -- backup url

In [348]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [349]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

### Submiting more than one jobs (test)

<br>

In [ ]:
urls = ['/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.9_EGFR_advanced.json', '/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.1_RHOBTB2.json']

In [ ]:
dict_workflows = {}
for i in urls:
    feature = (os.path.splitext(os.path.basename(i))[0])
    print(i, feature)
    with open(i,'r') as inf:
        query = json.load(inf)
        kcresult = submit_to_ars(query)
        #xx = retrieve_devars_results(kcresult)
        
        dict_workflows[feature] = kcresult

In [ ]:
dict_workflows

In [ ]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    print(keys, val)
    
    result_status = retrieve_ars_results(val)
    
    workflow_result_messages[keys] = result_status

In [ ]:
workflow_result_messages

In [ ]:
col = []
final_dict = defaultdict(list)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k].items():
        if key.startswith('kp-'):
            key_mod = key.replace('kp-','')
        else:
            key_mod = key
        
        final_dict[key_mod].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

In [ ]:
df

In [ ]:
df.loc['pk_id']

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
pwd

<br>

## Test for individual run

<br>

In [350]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.1_ChemSubstances_related_to_Disease.json') as inf:
    query1 = json.load(inf)

In [351]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "edges": {
                "e00": {
                    "subject": "n00",
                    "object": "n01",
                    "predicates": [
                        "biolink:has_real_world_evidence_of_association_with"
                    ]
                }
            },
            "nodes": {
                "n00": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                },
                "n01": {
                    "categories": [
                        "biolink:Disease"
                    ],
                    "ids": [
                        "MONDO:0005301"
                    ]
                }
            }
        }
    }
}


In [352]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=5d6b1b89-d156-4235-a836-b87c150255c8


In [353]:
retrieve_ars_results(kcresult)

Running
Running
ara-aragorn Running 0


{'pk_id': 'https://arax.ncats.io/?source=ARS&id=5d6b1b89-d156-4235-a836-b87c150255c8'}

In [354]:
kcresult = 'https://arax.ncats.io/?source=ARS&id=ca41b368-c9e3-433d-94e2-14f12d3e0a4a'
ars_url='https://ars.transltr.io/ars/api'
pk = 'https://arax.ncats.io/?source=ARS&id=' + kcresult
message_url = f'{ars_url}/messages/{kcresult}?trace=y'
response = requests.get(message_url)
j = response.json()

In [355]:
j.keys()

dict_keys(['message', 'status', 'actor', 'children'])

In [356]:
j

{'message': '5d6b1b89-d156-4235-a836-b87c150255c8',
 'status': 'Done',
 'actor': {'pk': 9,
  'channel': 'general',
  'agent': 'ars-default-agent',
  'path': ''},
 'children': [{'message': '07536318-c2c3-4e95-a979-dbdb016aab0a',
   'status': 'Done',
   'code': 200,
   'actor': {'pk': 1,
    'channel': 'general',
    'agent': 'ara-aragorn',
    'path': 'runquery'},
   'children': []},
  {'message': '02fd5b8c-4e31-423c-8f0d-a6170ad63596',
   'status': 'Running',
   'code': 200,
   'actor': {'pk': 2,
    'channel': 'general',
    'agent': 'ara-arax',
    'path': 'runquery'},
   'children': []}]}

In [361]:
dictionary_test = {}
for child in j['children']:
    print(child['actor']['agent'])
    dictionary_test
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        #print(child_response['fields']['data']['message']['knowledge_graph']['edges'])
        if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                #print(val)
                test_att_values =[]
                for tx in edge_ex[val]['attributes']:
                    print(tx['attribute_type_id'], tx['value'])
                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                        #print(tx['value'])
                        test_att_values.append(tx['value'])
                        dictionary_test[child['actor']['agent']] = test_att_values
        else:
            pass
    

ara-aragorn
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 6.229829111969533
biolink:has_confidence_level [6.036538523805004, 6.399444017494373]
biolink:has_evidence 1.03125
biolink:has_confidence_level [0.7157894736842105, 1.4866920152091254]
biolink:has_evidence 0.0938032973280273
biolink:has_confidence_level [0.07321668909825034, 0.11762936221419976]
biolink:has_count 330
biolink:has_count 320
biolink:has_count 3518
EDAM:operation_3438 0.6500301987807314
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 6.199057453302779
biolink:has_confidence_level [6.014486176774809, 6.364381433728163]
biolink:has_evidence 1.0
biolink:has_confidence_level [0.7047619047619048, 1.4189189189189189

biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 5.980575915094381
biolink:has_confidence_level [5.569279886675423, 6.296913243306538]
biolink:has_evidence 0.8037383177570093
biolink:has_confidence_level [0.3986013986013986, 1.5733333333333333]
biolink:has_evidence 0.0244457077885162
biolink:has_confidence_level [0.015343203230148048, 0.035499398315282794]
biolink:has_count 86
biolink:has_count 107
biolink:has_count 3518
EDAM:operation_3438 0.21735384771730706
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 5.966435158034026
biolink:has_confidence_level [5.560970049925861, 6.289208550297077]
biolink:has_evidence 0.7924528301886793
biolink:has_confidence_level [0.3971631205673759, 1.

biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 4.087162166773205
biolink:has_confidence_level [3.639137444246245, 4.42561656459841]
biolink:has_evidence 0.12100840336134454
biolink:has_confidence_level [0.06794682422451995, 0.19573643410852712]
biolink:has_evidence 0.02046617396247868
biolink:has_confidence_level [0.012382234185733513, 0.030385078219013237]
biolink:has_count 72
biolink:has_count 595
biolink:has_count 3518
EDAM:operation_3438 1.2086499008579226
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 3.894369310111955
biolink:has_confidence_level [3.630951859659805, 4.1154053790866]
biolink:has_evidence 0.09978991596638656
biolink:has_confidence_level [0.07125427037579307, 0.1345062429057889]
biolink:has_evidence 0.0540

biolink:p_value 1.6672806688862353e-141
biolink:has_evidence 3.4084037236653595
biolink:has_confidence_level [2.5329349863114596, 3.968019511600782]
biolink:has_evidence 0.061381074168797956
biolink:has_confidence_level [0.021834061135371178, 0.12804878048780488]
biolink:has_evidence 0.006822057987492893
biolink:has_confidence_level [0.0026917900403768506, 0.01263537906137184]
biolink:has_count 24
biolink:has_count 391
biolink:has_count 3518
EDAM:operation_3438 0.7942556491352062
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 4.4629936607565404e-268
biolink:p_value 8.822594084603003e-260
biolink:has_evidence 2.99023196428808
biolink:has_confidence_level [2.531924374805536, 3.3454352376153564]
biolink:has_evidence 0.04040404040404041
biolink:has_confidence_level [0.023626373626373626, 0.06258064516129032]
biolink:has_evidence 0

biolink:has_confidence_level [2.1331845651619905, 3.2662830299012695]
biolink:has_evidence 0.033393501805054154
biolink:has_confidence_level [0.015586546349466776, 0.059]
biolink:has_evidence 0.010517339397384877
biolink:has_confidence_level [0.0051144010767160165, 0.017749699157641397]
biolink:has_count 37
biolink:has_count 1108
biolink:has_count 3518
EDAM:operation_3438 2.2507295632782824
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 1.3121348906433504e-151
biolink:p_value 2.5938718278234934e-143
biolink:has_evidence 2.651391887985382
biolink:has_confidence_level [2.122547758716713, 3.032759444514486]
biolink:has_evidence 0.02879177377892031
biolink:has_confidence_level [0.01577437858508604, 0.04550499445061043]
biolink:has_evidence 0.015918135304150087
biolink:has_confidence_level [0.008882907133243608, 0.0246690734055355]

...

biolink:has_count 3518
EDAM:operation_3438 10.690965425571843
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 1.2310483241615423e-99
biolink:p_value 2.433577210317382e-91
biolink:has_evidence 2.123240968641721
biolink:has_confidence_level [1.650334579738024, 2.4741019427150337]
biolink:has_evidence 0.016978346456692914
biolink:has_confidence_level [0.010058479532163742, 0.0254149377593361]
biolink:has_evidence 0.01961341671404207
biolink:has_confidence_level [0.011574697173620458, 0.029482551143200964]
biolink:has_count 69
biolink:has_count 4064
biolink:has_count 3518
EDAM:operation_3438 8.25538352451529
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 5.875362963800341e

biolink:supporting_data_source infores:cohd
biolink:p_value 1.7066523405345581e-273
biolink:p_value 3.373767024692944e-265
biolink:has_evidence 1.7147122726338093
biolink:has_confidence_level [1.5201206232480404, 1.885278007868112]
biolink:has_evidence 0.01128427426290983
biolink:has_confidence_level [0.00911238609517381, 0.0136452925494598]
biolink:has_evidence 0.09323479249573621
biolink:has_confidence_level [0.07267833109017496, 0.11702767749699157]
biolink:has_count 328
biolink:has_count 29067
biolink:has_count 3518
EDAM:operation_3438 59.0450868373735
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 0.0
biolink:p_value 0.0
biolink:has_evidence 1.6843240204761238
biolink:has_confidence_level [1.5174784211676167, 1.8332184464145551]
biolink:has_evidence 0.010946522721566659
biolink:has_confidence_level [0.009113811499703616, 

biolink:p_value 6.670808469106605e-39
biolink:has_evidence 2.2307891492101843
biolink:has_confidence_level [1.3834912888229807, 2.748732240745039]
biolink:has_evidence 0.018906144496961513
biolink:has_confidence_level [0.0074580484773151025, 0.03466076696165192]
biolink:has_evidence 0.007959067652075043
biolink:has_confidence_level [0.003230148048452221, 0.014139590854392299]
biolink:has_count 28
biolink:has_count 1481
biolink:has_count 3518
EDAM:operation_3438 3.0084210137320726
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 1.1208945440925148e-216
biolink:p_value 2.2158215596698924e-208
biolink:has_evidence 1.5776165704490948
biolink:has_confidence_level [1.379396427587342, 1.750959984019825]
biolink:has_evidence 0.00983860955927995
biolink:has_confidence_level [0.00792393026941363, 0.01191868736366223]
biolink:has_evidence 

biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 7.571883512039105e-175
biolink:p_value 1.496835078885034e-166
biolink:has_evidence 1.481826277085548
biolink:has_confidence_level [1.2762637562521424, 1.6607637358952618]
biolink:has_evidence 0.008939897311990335
biolink:has_confidence_level [0.007149213883120736, 0.010887952511303171]
biolink:has_evidence 0.08413871517907902
biolink:has_confidence_level [0.0648721399730821, 0.10649819494584838]
biolink:has_count 296
biolink:has_count 33110
biolink:has_count 3518
EDAM:operation_3438 67.25781213009381
biolink:provided_by 3
biolink:aggregator_knowledge_source infores:aragorn
biolink:has_numeric_value 1
biolink:original_knowledge_source infores:cohd
biolink:supporting_data_source infores:cohd
biolink:p_value 2.6056348410679456e-38
biolink:p_value 5.1509054869561866e-30
biolink:has_evidence 2.2141072968394413
biolink:has_confidence_level [1.275837658246511, 2.767492535024228]
biolink:

biolink:publications []
biolink:original_knowledge_source infores:aragorn-ranker-ara
biolink:has_numeric_value 1
biolink:has_count 9
biolink:publications []
biolink:original_knowledge_source infores:aragorn-ranker-ara
biolink:has_numeric_value 1
biolink:has_count 537
biolink:publications []
biolink:original_knowledge_source infores:aragorn-ranker-ara
biolink:has_numeric_value 1
ara-arax


In [254]:
if tx['attribute_type_id'] == 'biolink:primary_knowledge_source':
    print(tx['value'])
else:
    print('Nothing')

Nothing


In [256]:
dictionary_test

{'ara-arax': ['infores:rtx-kg2', 'infores:semmeddb', 'infores:arax'],
 'ara-bte': [['infores:semmeddb'], ['SEMMED Gene API']],
 'ara-improving': ['infores:lincs',
  'infores:spoke',
  'infores:improving-agent']}

In [257]:
dictionary={}
results = {}
dictionary_2 = {}
for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                print(val)
                                test_att_values =[]
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source'):
                                        test_att_values.append(tx['value'])
                                        dictionary_2[child['actor']['agent']] = test_att_values
                    else:
                        pass 
                else:
                    dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)

ARS Error
ara-aragorn ARS Error 0
Done
BTE:NCBIGene:100616101-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:1869-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:5241-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:5555-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:NCBIGene:7332-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C0079427-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C1418864-biolink:entity_negatively_regulates_entity-NCBIGene:23221
BTE:UMLS:C4085573-biolink:entity_negatively_regulates_entity-NCBIGene:23221
N1_0
N1_1
N1_2
N1_3
N1_4
N1_5
N1_6
N1_7
N1_8
RTX-KG2:UniProtKB:Q01094-biolink:entity_negatively_regulates_entity-UniProtKB:Q9BYZ6
RTX-KG2:UniProtKB:Q13618-biolink:entity_negatively_regulates_entity-UniProtKB:Q9BYZ6
ara-arax Done 9
Done
NCBIGene:100616101-biolink:entity_negatively_regulates_entity-NCBIGene:23221
UMLS:C4085573-biolink:entity_negati

In [258]:
dictionary_2

{'ara-aragorn': [],
 'ara-arax': ['infores:rtx-kg2', 'infores:semmeddb', 'infores:arax'],
 'ara-bte': [['infores:semmeddb'], ['SEMMED Gene API']],
 'ara-unsecret': [],
 'kp-genetics': [],
 'kp-molecular': [],
 'ara-explanatory': [],
 'ara-improving': ['infores:lincs',
  'infores:spoke',
  'infores:improving-agent'],
 'kp-cam': [],
 'kp-textmining': [],
 'ara-aragorn-exp': [],
 'kp-openpredict': [],
 'kp-icees': [],
 'ara-robokop': [],
 'kp-chp': [],
 'kp-cohd': [],
 'kp-icees-dili': []}

In [231]:
for child in j['children']:
    #print(child['status'])
    if child['status']  == 'Done':
        childmessage_id = child['message']
        child_url = f'{ars_url}/messages/{childmessage_id}'
        child_response = requests.get(child_url).json()
        if child_response['fields']['data']['message']['knowledge_graph']['edges']:
            print('correct')
        else:
            print('incorrect')

correct
correct
incorrect
incorrect
incorrect
incorrect
correct
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect
incorrect


In [167]:
child_response['fields']['data']['message']['knowledge_graph']

{'nodes': {}, 'edges': {}}

In [126]:
mm = child_response['pk']
mm

'615f54a8-cb94-497f-9152-ec883597c17b'

In [128]:
xt = f'{ars_url}/messages/{childmessage_id}/{mm}'
xt

'https://ars.transltr.io/ars/api/messages/615f54a8-cb94-497f-9152-ec883597c17b/615f54a8-cb94-497f-9152-ec883597c17b'

In [94]:
f'{ars_url}/model/{mm}'

'https://ars.transltr.io/ars/api/model/615f54a8-cb94-497f-9152-ec883597c17b'

In [98]:
requests.get(xt).json

<bound method Response.json of <Response [200]>>

In [ ]:
childmessage_id = child['message']
child_url = f'{ars_url}/messages/{childmessage_id}'

 child_response = requests.get(child_url).json()

In [ ]:
result_status = retrieve_ars_results(kcresult)

In [ ]:
kcresult

In [ ]:
result_status 

In [ ]:
pwd

In [ ]:
df

In [ ]:
def make_hyperlink(value):
    #url = "https://custom.url/{}"
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [ ]:
make_hyperlink('https://arax.ncats.io/?source=ARS&id=325cfd91-ee6c-48eb-8a09-7226bbafb482')

In [ ]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [ ]:
df

In [ ]:
df.to_excel('test_ara_worklow.xlsx')

In [ ]:
df